In [2]:
###################importing libraries##############################
from __future__ import print_function
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from openpyxl import load_workbook
from tensorflow.keras.layers import LayerNormalization
from keras.callbacks import EarlyStopping


In [3]:
####################Reading the csv Files####################################
x_train=pd.read_excel(r'G:\Class_project\A_train_samples.xlsx')
y_train=pd.read_excel(r'G:\Class_project\A_train_labels.xlsx')
x_test=pd.read_excel(r'G:\Class_project\A_test_samples.xlsx')
y_test=pd.read_excel(r'G:\Class_project\A_test_labels.xlsx')
x_train=x_train[['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21',
       'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28', 'F29', 'F30', 'F31',
       'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F38', 'F39', 'F40', 'F41',
       'F42']]
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
type(y_train)

(175341, 42)
(175341, 25)
(175341, 42)
(175341, 1)


pandas.core.frame.DataFrame

In [4]:
####################scaling the training Data######################################
scaler = MinMaxScaler()
x_train_scl = scaler.fit_transform(x_train)
x_test_scl = scaler.transform(x_test)
#x_train_scl=to_dataframe(x_train_scl)
x_train = pd.DataFrame(data=x_train_scl)
x_test=pd.DataFrame(data=x_test_scl)

In [5]:
################Creating Function to fit and evaluate the data###################
def fit_and_evaluate(t_x, val_x, t_y, val_y, callbacks,EPOCHS=100, BATCH_SIZE=1000):
    results = model.fit(t_x, t_y,callbacks=callbacks, epochs=EPOCHS, batch_size=BATCH_SIZE, 
              verbose=1, validation_split=0.1)  
    print("Val Score: ", model.evaluate(val_x, val_y))
    return results

In [6]:
y_train=y_train[['B','C5']]

In [ ]:
#############Creating loop for each column in y_train#################
from pandas import ExcelWriter
import statistics

accuracy = []
f_score = []
y_pred_list = []

from keras.layers import GaussianNoise
t=y_train.columns
count=0
for i in range(y_train.shape[1]):
    temp = ""
    temp1 = ""
    y_train_1=y_train[t[i]]
    y_train_1=pd.DataFrame(y_train_1)
    model = Sequential()
    # in the first layer, you must specify the expected input data shape:
    # here, 43-dimensional vectors.
    model.add(Dense(64, activation='relu', input_dim=42))

    model.add(Dense(128, activation='relu'))

    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.1)))
    model.add(BatchNormalization())
    #model.add(Dropout(0.1))
    model.add(Dense(11, activation='softmax'))
    
    model.add(Dense(512, activation = 'linear'))

    #sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=10)
    n_folds=10
    #save the model history in a list after fitting so that we can plot later
    model_history = [] 

    for i in range(n_folds):
        print("Training on Fold: ",i+1)
        t_x, val_x, t_y, val_y = train_test_split(x_train, y_train_1, test_size=0.1,
                                                  random_state = np.random.randint(1,1000, 1)[0])
        model_history.append(fit_and_evaluate(t_x, val_x, t_y, val_y,[es], 50,1000))
        score = model.evaluate(x_test,y_test,batch_size=1000)
        print('score:',score)
        temp = str(round(score[1],2))
        print('temp',temp)

        y_pred=model.predict_classes(x_test)
        
        f1= f1_score(y_test, y_pred,average='micro')
        print(f1)
        temp1 = str(round(f1,2))
        print(temp1)
        y_pred_list.append(y_pred)
        print(y_pred_list)
        accuracy.append(temp)
        f_score.append(temp1)
    
df1 = pd.DataFrame({'B': y_pred_list[0], 'C5': y_pred_list[10], 'C10': y_pred_list[20], 'C15': y_pred_list[30], 'C20': y_pred_list[40], 'C25': y_pred_list[50], 'C30': y_pred_list[60], 'C35': y_pred_list[70], 'C40': y_pred_list[80], 'A5': y_pred_list[90], 'A10': y_pred_list[100], 'A15': y_pred_list[110], 'A20': y_pred_list[120], 'A25': y_pred_list[130], 'A30': y_pred_list[140], 'A35': y_pred_list[150], 'A40': y_pred_list[160], 'N5': y_pred_list[170], 'N10': y_pred_list[180], 'N15': y_pred_list[190], 'N20': y_pred_list[200], 'N25': y_pred_list[210], 'N30': y_pred_list[220], 'N35': y_pred_list[230], 'N40': y_pred_list[240]})
writer = ExcelWriter("a_prediction.xlsx")
df1.to_excel(writer, 'Sheet1', index=False)

df2 = pd.DataFrame({'B': y_pred_list[1], 'C5': y_pred_list[11], 'C11': y_pred_list[21], 'C15': y_pred_list[31], 'C21': y_pred_list[41], 'C25': y_pred_list[51], 'C31': y_pred_list[61], 'C35': y_pred_list[71], 'C41': y_pred_list[81], 'A5': y_pred_list[91], 'A11': y_pred_list[101], 'A15': y_pred_list[111], 'A21': y_pred_list[121], 'A25': y_pred_list[131], 'A31': y_pred_list[141], 'A35': y_pred_list[151], 'A41': y_pred_list[161], 'N5': y_pred_list[171], 'N11': y_pred_list[181], 'N15': y_pred_list[191], 'N21': y_pred_list[211], 'N25': y_pred_list[211], 'N31': y_pred_list[221], 'N35': y_pred_list[231], 'N41': y_pred_list[241]})
writer = ExcelWriter("a_prediction.xlsx")
df2.to_excel(writer, 'Sheet2', index=False)

df3 = pd.DataFrame({'B': y_pred_list[2], 'C5': y_pred_list[12], 'C12': y_pred_list[22], 'C15': y_pred_list[32], 'C22': y_pred_list[42], 'C25': y_pred_list[52], 'C32': y_pred_list[62], 'C35': y_pred_list[72], 'C42': y_pred_list[82], 'A5': y_pred_list[92], 'A12': y_pred_list[102], 'A15': y_pred_list[112], 'A22': y_pred_list[122], 'A25': y_pred_list[132], 'A32': y_pred_list[142], 'A35': y_pred_list[152], 'A42': y_pred_list[162], 'N5': y_pred_list[172], 'N12': y_pred_list[182], 'N15': y_pred_list[192], 'N22': y_pred_list[222], 'N25': y_pred_list[212], 'N32': y_pred_list[222], 'N35': y_pred_list[232], 'N42': y_pred_list[242]})
writer = ExcelWriter("a_prediction.xlsx")
df3.to_excel(writer, 'Sheet3', index=False)

df4 = pd.DataFrame({'B': y_pred_list[3], 'C5': y_pred_list[13], 'C13': y_pred_list[23], 'C15': y_pred_list[33], 'C23': y_pred_list[43], 'C25': y_pred_list[53], 'C33': y_pred_list[63], 'C35': y_pred_list[73], 'C43': y_pred_list[83], 'A5': y_pred_list[93], 'A13': y_pred_list[103], 'A15': y_pred_list[113], 'A23': y_pred_list[123], 'A25': y_pred_list[133], 'A33': y_pred_list[143], 'A35': y_pred_list[153], 'A43': y_pred_list[163], 'N5': y_pred_list[173], 'N13': y_pred_list[183], 'N15': y_pred_list[193], 'N23': y_pred_list[233], 'N25': y_pred_list[213], 'N33': y_pred_list[223], 'N35': y_pred_list[233], 'N43': y_pred_list[243]})
writer = ExcelWriter("a_prediction.xlsx")
df4.to_excel(writer, 'Sheet4', index=False)
writer.save()

df5 = pd.DataFrame({'B': y_pred_list[4], 'C5': y_pred_list[14	], 'C14': y_pred_list[24], 'C15': y_pred_list[34], 'C24': y_pred_list[44], 'C25': y_pred_list[54], 'C34': y_pred_list[64], 'C35': y_pred_list[74], 'C44': y_pred_list[84], 'A5': y_pred_list[94], 'A14': y_pred_list[104], 'A15': y_pred_list[114], 'A24': y_pred_list[124], 'A25': y_pred_list[134], 'A34': y_pred_list[144], 'A35': y_pred_list[154], 'A44': y_pred_list[164], 'N5': y_pred_list[174], 'N14': y_pred_list[184], 'N15': y_pred_list[194], 'N24': y_pred_list[244], 'N25': y_pred_list[214], 'N34': y_pred_list[224], 'N35': y_pred_list[234], 'N44': y_pred_list[244]})
writer = ExcelWriter("a_prediction.xlsx")
df5.to_excel(writer, 'Sheet5', index=False)


df6 = pd.DataFrame({'B': y_pred_list[5], 'C5': y_pred_list[15], 'C15': y_pred_list[25], 'C15': y_pred_list[35], 'C25': y_pred_list[45], 'C25': y_pred_list[55], 'C35': y_pred_list[65], 'C35': y_pred_list[75], 'C45': y_pred_list[85], 'A5': y_pred_list[95], 'A15': y_pred_list[105], 'A15': y_pred_list[115], 'A25': y_pred_list[125], 'A25': y_pred_list[135], 'A35': y_pred_list[145], 'A35': y_pred_list[155], 'A45': y_pred_list[165], 'N5': y_pred_list[175], 'N15': y_pred_list[185], 'N15': y_pred_list[195], 'N25': y_pred_list[255], 'N25': y_pred_list[215], 'N35': y_pred_list[225], 'N35': y_pred_list[235], 'N45': y_pred_list[245]})
writer = ExcelWriter("a_prediction.xlsx")
df6.to_excel(writer, 'Sheet6', index=False)

df7 = pd.DataFrame({'B': y_pred_list[6], 'C5': y_pred_list[16], 'C16': y_pred_list[26], 'C15': y_pred_list[36], 'C26': y_pred_list[46], 'C25': y_pred_list[56], 'C36': y_pred_list[66], 'C35': y_pred_list[76], 'C46': y_pred_list[86], 'A5': y_pred_list[96], 'A16': y_pred_list[106], 'A15': y_pred_list[116], 'A26': y_pred_list[126], 'A25': y_pred_list[136], 'A36': y_pred_list[146], 'A35': y_pred_list[156], 'A46': y_pred_list[166], 'N5': y_pred_list[176], 'N16': y_pred_list[186], 'N15': y_pred_list[196], 'N26': y_pred_list[266], 'N25': y_pred_list[216], 'N36': y_pred_list[226], 'N35': y_pred_list[236], 'N46': y_pred_list[246]})
writer = ExcelWriter("a_prediction.xlsx")
df7.to_excel(writer, 'Sheet7', index=False)


df8 = pd.DataFrame({'B': y_pred_list[7], 'C5': y_pred_list[17], 'C17': y_pred_list[27], 'C15': y_pred_list[37], 'C27': y_pred_list[47], 'C25': y_pred_list[57], 'C37': y_pred_list[67], 'C35': y_pred_list[77], 'C47': y_pred_list[87], 'A5': y_pred_list[97], 'A17': y_pred_list[107], 'A15': y_pred_list[117], 'A27': y_pred_list[127], 'A25': y_pred_list[137], 'A37': y_pred_list[147], 'A35': y_pred_list[157], 'A47': y_pred_list[167], 'N5': y_pred_list[177], 'N17': y_pred_list[187], 'N15': y_pred_list[197], 'N27': y_pred_list[277], 'N25': y_pred_list[217], 'N37': y_pred_list[227], 'N35': y_pred_list[237], 'N47': y_pred_list[247]})
writer = ExcelWriter("a_prediction.xlsx")
df7.to_excel(writer, 'Sheet8', index=False)

df9 = pd.DataFrame({'B': y_pred_list[8], 'C5': y_pred_list[18], 'C18': y_pred_list[28], 'C15': y_pred_list[38], 'C28': y_pred_list[48], 'C25': y_pred_list[58], 'C38': y_pred_list[68], 'C35': y_pred_list[78], 'C48': y_pred_list[88], 'A5': y_pred_list[98], 'A18': y_pred_list[108], 'A15': y_pred_list[118], 'A28': y_pred_list[128], 'A25': y_pred_list[138], 'A38': y_pred_list[148], 'A35': y_pred_list[158], 'A48': y_pred_list[168], 'N5': y_pred_list[178], 'N18': y_pred_list[188], 'N15': y_pred_list[198], 'N28': y_pred_list[288], 'N25': y_pred_list[218], 'N38': y_pred_list[228], 'N35': y_pred_list[238], 'N48': y_pred_list[248]})
writer = ExcelWriter("a_prediction.xlsx")
df9.to_excel(writer, 'Sheet9', index=False)

df10 = pd.DataFrame({'B': y_pred_list[9], 'C5': y_pred_list[19], 'C19': y_pred_list[29], 'C15': y_pred_list[39], 'C29': y_pred_list[49], 'C25': y_pred_list[59], 'C39': y_pred_list[69], 'C35': y_pred_list[79], 'C49': y_pred_list[89], 'A5': y_pred_list[99], 'A19': y_pred_list[109], 'A15': y_pred_list[119], 'A29': y_pred_list[129], 'A25': y_pred_list[139], 'A39': y_pred_list[149], 'A35': y_pred_list[159], 'A49': y_pred_list[169], 'N5': y_pred_list[179], 'N19': y_pred_list[189], 'N15': y_pred_list[199], 'N29': y_pred_list[299], 'N25': y_pred_list[219], 'N39': y_pred_list[229], 'N35': y_pred_list[239], 'N49': y_pred_list[249]})
writer = ExcelWriter("a_prediction.xlsx")
df10.to_excel(writer, 'Sheet10', index=False)

writer.save()


print("accuracy")    
print(accuracy)
print("f_score")
print(f_score)

data_a = []
for i in range(250):
    data_a.append(list(map(float, accuracy[i].split(",")[:250])))
len(data_a)
data_b = []

#for i in range(len(data_a)):
 #   temp = data_a[i]
  #  temp.append(statistics.mean(temp))
   # temp.append(statistics.stdev(temp))
    #data_b.append(temp)

df11 = pd.DataFrame(data_a)#, 'N5': data_b[4], 'N6': data_b[5], 'N7': data_b[6], 'N8': data_b[7], 'N9': data_b[8], 'N10': data_b[9], 'N11': data_b[10], 'N12': data_b[11], 'N13': data_b[12], 'N14': data_b[13], 'N15': data_b[14], 'N16': data_b[15], 'N17': data_b[16], 'N18': data_b[17], 'N19': data_b[18], 'N20': data_b[19], 'N21': data_b[20], 'N22': data_b[21], 'N23': data_b[22], 'N24': data_b[23], 'N25': data_b[24]})
writer = ExcelWriter("performance_a_a.xlsx")
df11.to_excel(writer, 'Sheet1', index=False)
writer.save()


data_b = []
for i in range(250):
    data_b.append(list(map(float, f_score[i].split(",")[:250])))

#data_b = []

#for i in range(len(data_a)):
 #   temp = data_a[i]
  #  temp.append(statistics.mean(temp))
   # temp.append(statistics.stdev(temp))
    #data_b.append(temp)

df11 = pd.DataFrame(data_b)#, 'N3': data_b[2], 'N4': data_b[3], 'N5': data_b[4], 'N6': data_b[5], 'N7': data_b[6], 'N8': data_b[7], 'N9': data_b[8], 'N10': data_b[9], 'N11': data_b[10], 'N12': data_b[11], 'N13': data_b[12], 'N14': data_b[13], 'N15': data_b[14], 'N16': data_b[15], 'N17': data_b[16], 'N18': data_b[17], 'N19': data_b[18], 'N20': data_b[19], 'N21': data_b[20], 'N22': data_b[21], 'N23': data_b[22], 'N24': data_b[23], 'N25': data_b[24]})
writer = ExcelWriter("performance_a_f.xlsx")
df11.to_excel(writer, 'Sheet1', index=False)
writer.save()

Training on Fold:  1
Train on 142025 samples, validate on 15781 samples
Epoch 1/50
142025/142025 [==============================] - 3s 20us/step - loss: 6.7767 - accuracy: 0.6172 - val_loss: 2.8038 - val_accuracy: 0.5071
Epoch 2/50
142025/142025 [==============================] - 2s 17us/step - loss: 1.4757 - accuracy: 0.7090 - val_loss: 2.8018 - val_accuracy: 0.5071
Epoch 3/50
142025/142025 [==============================] - 2s 16us/step - loss: 1.0432 - accuracy: 0.7119 - val_loss: 1.5325 - val_accuracy: 0.5611
Epoch 4/50
142025/142025 [==============================] - 2s 17us/step - loss: 0.9277 - accuracy: 0.7227 - val_loss: 1.6797 - val_accuracy: 0.5351
Epoch 5/50
142025/142025 [==============================] - 2s 17us/step - loss: 1.0716 - accuracy: 0.6897 - val_loss: 1.0327 - val_accuracy: 0.7071
Epoch 6/50
142025/142025 [==============================] - 3s 18us/step - loss: 0.9203 - accuracy: 0.7164 - val_loss: 0.9610 - val_accuracy: 0.7265
Epoch 7/50
142025/142025 [========

142025/142025 [==============================] - 3s 19us/step - loss: 0.8330 - accuracy: 0.7321 - val_loss: 0.8152 - val_accuracy: 0.7403
Epoch 12/50
142025/142025 [==============================] - 3s 19us/step - loss: 0.7450 - accuracy: 0.7638 - val_loss: 0.8128 - val_accuracy: 0.7474
Epoch 13/50
142025/142025 [==============================] - 3s 20us/step - loss: 0.7217 - accuracy: 0.7680 - val_loss: 0.7504 - val_accuracy: 0.7560
Epoch 14/50
142025/142025 [==============================] - 3s 20us/step - loss: 0.7128 - accuracy: 0.7706 - val_loss: 0.7363 - val_accuracy: 0.7641
Epoch 15/50
142025/142025 [==============================] - 3s 21us/step - loss: 0.8398 - accuracy: 0.7371 - val_loss: 1.0986 - val_accuracy: 0.6446
Epoch 16/50
142025/142025 [==============================] - 3s 20us/step - loss: 0.7910 - accuracy: 0.7431 - val_loss: 0.8177 - val_accuracy: 0.7397
Epoch 17/50
142025/142025 [==============================] - 3s 20us/step - loss: 0.8336 - accuracy: 0.7511 - va

17535/17535 [==============================] - 1s 35us/step
Val Score:  [1.0031548031959774, 0.6591958999633789]
175341/175341 [==============================] - 1s 8us/step
score: [1.1390501818856884, 0.5770469903945923]
temp 0.58
0.5770470112523597
0.58
[array([4, 4, 4, ..., 6, 6, 6], dtype=int64), array([4, 4, 4, ..., 6, 6, 6], dtype=int64), array([4, 4, 4, ..., 6, 6, 6], dtype=int64), array([4, 4, 7, ..., 7, 6, 6], dtype=int64)]
Training on Fold:  5
Train on 142025 samples, validate on 15781 samples
Epoch 1/50
142025/142025 [==============================] - 3s 19us/step - loss: 0.8835 - accuracy: 0.7481 - val_loss: 0.8775 - val_accuracy: 0.7432
Epoch 2/50
142025/142025 [==============================] - 3s 20us/step - loss: 0.8384 - accuracy: 0.7580 - val_loss: 2.2817 - val_accuracy: 0.6108
Epoch 3/50
142025/142025 [==============================] - 3s 23us/step - loss: 1.5795 - accuracy: 0.6747 - val_loss: 1.2073 - val_accuracy: 0.7107
Epoch 4/50
142025/142025 [==================

0.6976006752556447
0.7
[array([4, 4, 4, ..., 6, 6, 6], dtype=int64), array([4, 4, 4, ..., 6, 6, 6], dtype=int64), array([4, 4, 4, ..., 6, 6, 6], dtype=int64), array([4, 4, 7, ..., 7, 6, 6], dtype=int64), array([4, 4, 4, ..., 6, 6, 6], dtype=int64)]
Training on Fold:  6
Train on 142025 samples, validate on 15781 samples
Epoch 1/50
142025/142025 [==============================] - 3s 20us/step - loss: 0.7278 - accuracy: 0.7639 - val_loss: 0.7284 - val_accuracy: 0.7558
Epoch 2/50
142025/142025 [==============================] - 3s 22us/step - loss: 0.7043 - accuracy: 0.7721 - val_loss: 0.7322 - val_accuracy: 0.7543
Epoch 3/50
142025/142025 [==============================] - 3s 21us/step - loss: 0.6900 - accuracy: 0.7747 - val_loss: 0.7343 - val_accuracy: 0.7683
Epoch 4/50
142025/142025 [==============================] - 3s 20us/step - loss: 0.7000 - accuracy: 0.7710 - val_loss: 0.6877 - val_accuracy: 0.7695
Epoch 5/50
142025/142025 [==============================] - 3s 20us/step - loss: 1.

175341/175341 [==============================] - 1s 8us/step
score: [1.4853877302973348, 0.6827325224876404]
temp 0.68
0.6827325040920263
0.68
[array([4, 4, 4, ..., 6, 6, 6], dtype=int64), array([4, 4, 4, ..., 6, 6, 6], dtype=int64), array([4, 4, 4, ..., 6, 6, 6], dtype=int64), array([4, 4, 7, ..., 7, 6, 6], dtype=int64), array([4, 4, 4, ..., 6, 6, 6], dtype=int64), array([4, 4, 4, ..., 6, 6, 6], dtype=int64), array([4, 4, 4, ..., 6, 6, 6], dtype=int64), array([4, 4, 4, ..., 6, 6, 6], dtype=int64)]
Training on Fold:  9
Train on 142025 samples, validate on 15781 samples
Epoch 1/50
142025/142025 [==============================] - 3s 20us/step - loss: 1.1914 - accuracy: 0.7389 - val_loss: 1.2477 - val_accuracy: 0.7372
Epoch 2/50
142025/142025 [==============================] - 3s 21us/step - loss: 1.1744 - accuracy: 0.7463 - val_loss: 1.2587 - val_accuracy: 0.7431
Epoch 3/50
142025/142025 [==============================] - 3s 20us/step - loss: 1.3227 - accuracy: 0.7094 - val_loss: 1.2164 

...

142025/142025 [==============================] - 3s 20us/step - loss: 1.0275 - accuracy: 0.6987 - val_loss: 1.0479 - val_accuracy: 0.6965
Epoch 40/50
142025/142025 [==============================] - 3s 20us/step - loss: 1.0133 - accuracy: 0.7029 - val_loss: 1.0420 - val_accuracy: 0.6910
Epoch 41/50
142025/142025 [==============================] - 3s 23us/step - loss: 1.0045 - accuracy: 0.7038 - val_loss: 1.0662 - val_accuracy: 0.6897
Epoch 42/50
142025/142025 [==============================] - 4s 26us/step - loss: 0.9998 - accuracy: 0.7096 - val_loss: 1.0646 - val_accuracy: 0.6896
Epoch 43/50
142025/142025 [==============================] - 4s 28us/step - loss: 1.0023 - accuracy: 0.7045 - val_loss: 1.0403 - val_accuracy: 0.6910
Epoch 44/50
142025/142025 [==============================] - 3s 22us/step - loss: 0.9881 - accuracy: 0.7134 - val_loss: 1.0387 - val_accuracy: 0.6932
Epoch 45/50
142025/142025 [==============================] - 3s 23us/step - loss: 0.9770 - accuracy: 0.7171 - va

142025/142025 [==============================] - 3s 20us/step - loss: 0.9484 - accuracy: 0.7281 - val_loss: 0.9563 - val_accuracy: 0.7248
Epoch 10/50
142025/142025 [==============================] - 3s 19us/step - loss: 0.9559 - accuracy: 0.7259 - val_loss: 0.9683 - val_accuracy: 0.7202
Epoch 11/50
142025/142025 [==============================] - 3s 19us/step - loss: 0.9477 - accuracy: 0.7273 - val_loss: 0.9520 - val_accuracy: 0.7251
Epoch 12/50
142025/142025 [==============================] - 3s 19us/step - loss: 0.9488 - accuracy: 0.7275 - val_loss: 0.9929 - val_accuracy: 0.6928
Epoch 13/50
142025/142025 [==============================] - 3s 19us/step - loss: 0.9582 - accuracy: 0.7226 - val_loss: 0.9672 - val_accuracy: 0.7228
Epoch 14/50
142025/142025 [==============================] - 3s 20us/step - loss: 0.9430 - accuracy: 0.7294 - val_loss: 0.9641 - val_accuracy: 0.7210
Epoch 15/50
142025/142025 [==============================] - 3s 20us/step - loss: 0.9444 - accuracy: 0.7284 - va

142025/142025 [==============================] - 3s 20us/step - loss: 0.9247 - accuracy: 0.7364 - val_loss: 0.8881 - val_accuracy: 0.7427
Epoch 2/50
142025/142025 [==============================] - 3s 19us/step - loss: 0.9231 - accuracy: 0.7357 - val_loss: 0.8917 - val_accuracy: 0.7431
Epoch 3/50
142025/142025 [==============================] - 3s 19us/step - loss: 0.9246 - accuracy: 0.7356 - val_loss: 0.8917 - val_accuracy: 0.7391
Epoch 4/50
142025/142025 [==============================] - 3s 20us/step - loss: 0.9237 - accuracy: 0.7357 - val_loss: 0.8998 - val_accuracy: 0.7415
Epoch 5/50
142025/142025 [==============================] - 3s 20us/step - loss: 0.9269 - accuracy: 0.7348 - val_loss: 0.8912 - val_accuracy: 0.7446
Epoch 6/50
142025/142025 [==============================] - 3s 19us/step - loss: 0.9260 - accuracy: 0.7357 - val_loss: 0.8936 - val_accuracy: 0.7473
Epoch 7/50
142025/142025 [==============================] - 3s 20us/step - loss: 0.9215 - accuracy: 0.7371 - val_loss

Epoch 8/50
142025/142025 [==============================] - 3s 19us/step - loss: 0.9669 - accuracy: 0.7287 - val_loss: 0.9934 - val_accuracy: 0.7282
Epoch 9/50
142025/142025 [==============================] - 3s 19us/step - loss: 0.9721 - accuracy: 0.7246 - val_loss: 0.9819 - val_accuracy: 0.7111
Epoch 10/50
142025/142025 [==============================] - 3s 19us/step - loss: 0.9692 - accuracy: 0.7188 - val_loss: 0.9465 - val_accuracy: 0.7252
Epoch 11/50
142025/142025 [==============================] - 3s 19us/step - loss: 0.9644 - accuracy: 0.7192 - val_loss: 0.9475 - val_accuracy: 0.7214
Epoch 12/50
142025/142025 [==============================] - 3s 20us/step - loss: 1.0279 - accuracy: 0.7051 - val_loss: 1.3030 - val_accuracy: 0.6157
Epoch 13/50
142025/142025 [==============================] - 3s 20us/step - loss: 1.0035 - accuracy: 0.7155 - val_loss: 0.9857 - val_accuracy: 0.7161
Epoch 14/50
142025/142025 [==============================] - 3s 19us/step - loss: 0.9862 - accuracy: 0

142025/142025 [==============================] - 3s 20us/step - loss: 0.9086 - accuracy: 0.7407 - val_loss: 0.9184 - val_accuracy: 0.7324
Epoch 3/50
142025/142025 [==============================] - 3s 21us/step - loss: 0.9095 - accuracy: 0.7403 - val_loss: 0.9273 - val_accuracy: 0.7316
Epoch 4/50
142025/142025 [==============================] - 3s 20us/step - loss: 0.9106 - accuracy: 0.7399 - val_loss: 0.9141 - val_accuracy: 0.7363
Epoch 5/50
142025/142025 [==============================] - 3s 19us/step - loss: 0.9093 - accuracy: 0.7406 - val_loss: 0.9261 - val_accuracy: 0.7270
Epoch 6/50
142025/142025 [==============================] - 3s 19us/step - loss: 0.9082 - accuracy: 0.7406 - val_loss: 0.9148 - val_accuracy: 0.7387
Epoch 7/50
142025/142025 [==============================] - 3s 20us/step - loss: 0.9100 - accuracy: 0.7403 - val_loss: 0.9384 - val_accuracy: 0.7256
Epoch 8/50
142025/142025 [==============================] - 3s 20us/step - loss: 0.9105 - accuracy: 0.7406 - val_loss

In [15]:
print(len(val_x))

17535
